In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install chromadb llama-index
!pip install requests
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 45.9 MB/s eta 0:00:00
   

In [3]:
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 14.0 MB/s eta 0:00:00


In [4]:
!pip install langdetect
!pip install llama-index-llms-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=9a9454229335bff6c81904d5689be8b8106e5dc818d46c0c198f9173c00bae1d
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [60]:
import os
import uuid
import requests
from langdetect import detect
from bs4 import BeautifulSoup
from typing import List, Optional, Dict, Any
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex, Document,StorageContext
from llama_index.core import QueryBundle
from llama_index.core.retrievers import VectorIndexRetriever
import numpy as np
from tqdm import tqdm
from IPython.display import Markdown, display
import chromadb
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.llms.together import TogetherLLM
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
from llama_index.core.vector_stores import VectorStoreQuery

In [6]:
def create_or_load_chroma_db(persist_directory: str, collection_name: str):
    """Initialize or load a Chroma database and return the collection and index if it exists."""
    db = chromadb.PersistentClient(path=persist_directory)

    # Check if the collection exists
    try:
        chroma_collection = db.get_collection(name=collection_name)
        if chroma_collection is not None:
            print(f"Collection '{collection_name}' found.")
            return chroma_collection, True  # Collection exists
    except Exception as e:
        print(f"Error checking for collection: {e}")

    # If not found, create a new collection
    chroma_collection = db.get_or_create_collection(name=collection_name)
    print(f"Created new collection: '{collection_name}'")
    return chroma_collection, False  # New collection created


In [63]:
persist_directory = '/content/drive/MyDrive/rag-model/chromaDB_llama'
collection_name = 'rag_llama'
embedding_model_name = 'sentence-transformers/all-mpnet-base-v2'

try:
    chroma_collection, exists = create_or_load_chroma_db(persist_directory, collection_name)
    embed_model = HuggingFaceEmbedding(model_name=embedding_model_name)

    # If the collection exists, load the existing index; otherwise, ingest documents
    if exists:
        # Load existing index here if needed (depends on how you've implemented index loading)
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        index = VectorStoreIndex.from_vector_store(vector_store,embed_model=embed_model,) # Example function, implement as needed
        print("Loaded existing index.")
    else:
        print("no index.")


except Exception as e:
    print(f"Execution failed: {e}")
    raise

Collection 'rag_llama' found.
Loaded existing index.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [64]:
vector_store

ChromaVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, collection_name=None, host=None, port=None, ssl=False, headers=None, persist_dir=None, collection_kwargs={})

In [65]:
index

In [66]:
query = "How does the ASEAN Human Rights Declaration emphasize the universality, indivisibility, interdependence, and interrelatedness of human rights?"


In [67]:
def initialize_llm(model_name: str, api_key: str) -> TogetherLLM:
    """Initialize the Together LLM"""
    return TogetherLLM(
        model=model_name,
        api_key=api_key
    )

In [68]:
model_name = "meta-llama/Llama-2-13b-chat-hf"
api_key = "481dc4b7f7e5a6bff243a6d6d6f7c896e83a909a5b52c5f3aaf663d291c8946a"
together_llm = initialize_llm(model_name=model_name, api_key=api_key)

In [69]:
def generate_response(query_text: str, llm: TogetherLLM) -> str:
    """Generate a response using Together AI based on retrieved documents"""
    try:
        print("Preparing to generate response")
        prompt = create_prompt(query_text)

        print("Sending request to LLM")
        response = llm.complete(prompt)
        print("Response generated successfully")
        # Extract the answer from the response
        answer = response.text.strip()  # Remove any extra spaces or newline characters

        return answer

        return response

    except Exception as e:
        print(f"Failed to generate response: {str(e)}")
        return "An error occurred while generating the response."

def create_prompt(query_and_document: str) -> str:
    """Create the prompt for the LLM with both the question and the document"""
    return f"""
    You are an AI assistant that helps users by providing detailed and comprehensive answers based on provided information.

    Here is the question and the corresponding feedback document:

    {query_and_document}

    Your task is to provide a complete and detailed answer based on the feedback document provided above.
    Make sure to include all relevant details and explanations found in the document to answer the question thoroughly.
    """


In [70]:
class LLamaRetriever(BaseRetriever):
    """Retriever over a Pinecone vector store with similarity threshold and MMR diversity."""

    def __init__(
        self,
        vector_store: VectorStoreIndex,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 4,
        similarity_threshold: float = 0.55   ) -> None:
        """Initialize retriever parameters."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        self._similarity_threshold = similarity_threshold
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve documents based on similarity score and diversity (MMR)."""

        # Generate embedding if not provided
        if query_bundle.embedding is None:
            query_embedding = self._embed_model.get_query_embedding(query_bundle.query_str)
        else:
            query_embedding = query_bundle.embedding

        # Define the query with a similarity threshold and MMR diversity parameter
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode
        )
        query_result = self._vector_store.query(vector_store_query)

        # Compile results that meet the similarity threshold and MMR diversity
        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = query_result.similarities[index]
            if score >= self._similarity_threshold:
                nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores[:self._similarity_top_k]


In [71]:
retriever = LLamaRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=4
)

In [72]:
retrieved_nodes=retriever.retrieve(query)

In [75]:
for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** eb3622d5-cd92-4872-88ca-e59a1b5429e1<br>**Similarity:** 0.7524034932923302<br>**Text:** aw. Every person is equal before the law. Every person is entitled without discrimination to equal protection of the law.4. The rights of women, children, the elderly,personswith disabilities, migrant workers, and vulnerable andmarginalisedgroups are an inalienable, integral and indivisible part of human rights and fundamental freedoms.5. Every person has the right to an effective and enforceable remedy, to be determined by a court or other competent authorities, for acts violating the rights granted to that person by the constitution or by law.6. The enjoyment of human rights and fundamental freedoms must be balanced with the performance of corresponding duties as every person has responsibilities to all other individuals, the community and the society where one lives. It is ultimately the primary responsibility of all ASEAN Member States to promote and protect all human rights and fundamental freedoms.7. All human rights are universal, indivisible, interdependent and interrelated. Al<br>

**Node ID:** 305fd211-1d5a-40d4-90de-fdd29b2f54cd<br>**Similarity:** 0.7438133827157664<br>**Text:** aw. Every person is equal before the law. Every person is entitled without discrimination to equal protection of the law.4. The rights of women, children, the elderly,personswith disabilities, migrant workers, and vulnerable andmarginalisedgroups are an inalienable, integral and indivisible part of human rights and fundamental freedoms.5. Every person has the right to an effective and enforceable remedy, to be determined by a court or other competent authorities, for acts violating the rights granted to that person by the constitution or by law.6. The enjoyment of human rights and fundamental freedoms must be balanced with the performance of corresponding duties as every person has responsibilities to all other individuals, the community and the society where one lives. It is ultimately the primary responsibility of all ASEAN Member States to promote and protect all human rights and fundamental freedoms.7. All human rights are universal, indivisible, interdependent and interrelated. Al<br>

**Node ID:** 6000b367-f319-43ac-adee-67595e8ef8de<br>**Similarity:** 0.7428741021122048<br>**Text:** l human rights and fundamental freedoms in this Declaration must be treated in a fair and equal manner, on the same footing and with the same emphasis. At the same time, therealisationof human rights must be considered in the regional and national context bearing in mind different political, economic, legal, social, cultural, historical and religious backgrounds.8. The human rights and fundamental freedoms of every person shall be exercised with due regard to the human rights and fundamental freedoms of others. The exercise of human rights and fundamental freedoms shall be subject only to such limitations as are determined by law solely for the purpose of securing due recognition for the human rights and fundamental freedoms of others, and to meet the just requirements of national security, public order, public health, public safety, public morality, as well as the general welfare of the peoples in a democratic society.9. In therealisationof the human rights and freedoms contained in t<br>

**Node ID:** 1b7398e4-02b9-4438-b739-56ee4d6bf2f5<br>**Similarity:** 0.7417712074334655<br>**Text:** l human rights and fundamental freedoms in this Declaration must be treated in a fair and equal manner, on the same footing and with the same emphasis. At the same time, therealisationof human rights must be considered in the regional and national context bearing in mind different political, economic, legal, social, cultural, historical and religious backgrounds.8. The human rights and fundamental freedoms of every person shall be exercised with due regard to the human rights and fundamental freedoms of others. The exercise of human rights and fundamental freedoms shall be subject only to such limitations as are determined by law solely for the purpose of securing due recognition for the human rights and fundamental freedoms of others, and to meet the just requirements of national security, public order, public health, public safety, public morality, as well as the general welfare of the peoples in a democratic society.9. In therealisationof the human rights and freedoms contained in t<br>

In [76]:
# Initialize a list to store the document outputs in paragraph form
documents = []

# Add the query at the start of the formatted output
formatted_output = f"Question: {query}\n\n"

# Loop through the nodes and extract the text for each document
for i, node_with_score in enumerate(retrieved_nodes, 1):
    # Extract the text content from the node
    document_text = node_with_score.node.text
    # Format each document as "Document X: <text>" in a paragraph
    formatted_document = f"Document {i}: {document_text}"
    # Append the formatted document to the list
    documents.append(formatted_document)

# Combine all documents into a single paragraph
formatted_output += " ".join(documents)

# Print the formatted output
print(formatted_output)

Question: How does the ASEAN Human Rights Declaration emphasize the universality, indivisibility, interdependence, and interrelatedness of human rights?

Document 1: aw. Every person is equal before the law. Every person is entitled without discrimination to equal protection of the law.4. The rights of women, children, the elderly,personswith disabilities, migrant workers, and vulnerable andmarginalisedgroups are an inalienable, integral and indivisible part of human rights and fundamental freedoms.5. Every person has the right to an effective and enforceable remedy, to be determined by a court or other competent authorities, for acts violating the rights granted to that person by the constitution or by law.6. The enjoyment of human rights and fundamental freedoms must be balanced with the performance of corresponding duties as every person has responsibilities to all other individuals, the community and the society where one lives. It is ultimately the primary responsibility of all AS

In [77]:
response = generate_response(query_text=formatted_output, llm=together_llm)

Preparing to generate response
Sending request to LLM
Response generated successfully


In [78]:
response

'The ASEAN Human Rights Declaration emphasizes the universality, indivisibility, interdependence, and interrelatedness of human rights in several ways:\n\n1. All human rights are universal and inalienable: The Declaration states that all human rights and fundamental freedoms are universal, indivisible, interdependent, and interrelated (Article 7). This means that all individuals are entitled to the same human rights and freedoms, regardless of their race, gender, religion, or any other status.\n\n2. The rights of vulnerable groups are an integral part of human rights: The Declaration emphasizes that the rights of women, children, the elderly, persons with disabilities, migrant workers, and vulnerable and marginalized groups are an inalienable, integral, and indivisible part of human rights and fundamental freedoms (Article 4). This highlights the importance of protecting the rights of these groups, who may be more vulnerable to human rights abuses.\n\n3. The enjoyment of human rights m